# Load fit and configs

In [1]:
import sys
sys.path

['/home/localdmonte/fs-ensembles',
 '/home/localdmonte/Fitsnap_4_python_path/FitSNAP',
 '/home/localdmonte/.conda/envs/Fitnsap_NN_23/lib/python310.zip',
 '/home/localdmonte/.conda/envs/Fitnsap_NN_23/lib/python3.10',
 '/home/localdmonte/.conda/envs/Fitnsap_NN_23/lib/python3.10/lib-dynload',
 '',
 '/home/localdmonte/.local/lib/python3.10/site-packages',
 '/home/localdmonte/.conda/envs/Fitnsap_NN_23/lib/python3.10/site-packages']

In [2]:
# !export PYTHONPATH
# !python -m pip install fitsnap3 --no-deps

In [ ]:
import numpy as np
import pickle
import torch
from pathlib import Path

import glob
list_of_weights=sorted(glob.glob('/home/localdmonte/Nickel_potential_EM_AL_correct/NN_ensembles_using_fitsnap/AL_006/NN_*/Ni_Pytorch.pt'))
print('These are the weights of the nn',list_of_weights)

print('Loading config file')

import time

t0 = time.time()

with open(r"/home/localdmonte/Ni_pot_EM_AL/NN_ensembles_using_fitsnap/evaluating_fit_Initial_fit/NN_1/configs.pickle", "rb") as file:
    configs = pickle.load(file)

t1 = time.time()
total = t1-t0

print('Total time (s) to load config file',total)

These are the weights of the nn ['/home/localdmonte/Nickel_potential_EM_AL_correct/NN_ensembles_using_fitsnap/AL_006/NN_1/Ni_Pytorch.pt', '/home/localdmonte/Nickel_potential_EM_AL_correct/NN_ensembles_using_fitsnap/AL_006/NN_2/Ni_Pytorch.pt', '/home/localdmonte/Nickel_potential_EM_AL_correct/NN_ensembles_using_fitsnap/AL_006/NN_3/Ni_Pytorch.pt', '/home/localdmonte/Nickel_potential_EM_AL_correct/NN_ensembles_using_fitsnap/AL_006/NN_4/Ni_Pytorch.pt', '/home/localdmonte/Nickel_potential_EM_AL_correct/NN_ensembles_using_fitsnap/AL_006/NN_5/Ni_Pytorch.pt']
Loading config file


# Make settings / input

In [ ]:
# # Can use input script path (string) or dictionary.
settings = "/home/localdmonte/Nickel_potential_EM_AL_correct/Evaluating_fitted_potentials/Evaluating_fitted_potential_005/NN_Ni_eval.in"

# settings = \
# {
# "BISPECTRUM":
#     {
#     "numTypes": 1,
#     "twojmax": 8,
#     "rcutfac": 4.1,
#     "rfac0": 0.99363,
#     "rmin0": 0.0,
#     "wj": 1.0,
#     "radelem": 0.5,
#     "type": "Ni",
#     "wselfallflag": 0,
#     "chemflag": 0,
#     "bzeroflag": 1,
#     "bikflag": 1,
#     "dgradflag": 1
#     },
# "CALCULATOR":
#     {
#     "calculator": "LAMMPSSNAP",
#     "energy": 1,
#     "force": 1,
#     "per_atom_energy": 1,
#     "nonlinear": 1
#     },
# "PYTORCH":
#     {
#     "layer_sizes": "num_desc 55 64 32 16 1",
#     "learning_rate": 1.5e-4,
#     "num_epochs": 1000,
#     "batch_size": 4,
#     "save_state_output": "Ni_Pytorch.pt"
#     },
# "SOLVER":
#     {
#     "solver": "PYTORCH"
#     },
# "SCRAPER":
#     {
#     "scraper": "JSON" 
#     },
# "PATH":
#     {
#     "dataPath": "/home/localdmonte/Ni_pot_EM_AL/bispectrum_df_and_json_files/EM_Ni_set/Entropy_set_rescaled_json_internet_factor"
#     },
# "OUTFILE":
#     {
#     "metrics": "Ni_metrics_eval.dat",
#     "potential": "Ni_pot"   
#     },
# "REFERENCE":
#     {
#     "units": "metal",
#     "atom_style": "atomic",
#     "pair_style": "zero 10.0",
#     "pair_coeff": "* *"
#     },
# "GROUPS":
#     {
#     "group_sections": "name training_size testing_size eweight fweight",
#     "group_types": "str float float float float",
#     "smartweights": 0,
#     "random_sampling": 0
#     }
# }

# # Add all the groups.
# chars = [chr(i) for i in range(ord('a'), ord('z')+1, 1)]
# for c in chars:
#     settings['GROUPS'][f'json-xa{c}'] = "1  0  1.0e6 1.0e6"

# Create FitSnap instance

# Calculate model energies/forces.

In [ ]:
from fitsnap3lib.fitsnap import FitSnap
import pickle

#from mpi4py import MPI
#comm = MPI.COMM_WORLD

# For settings can use dictionary or string path to input script.
#settings = "NN_Ni_eval.in"

fs = FitSnap(settings, arglist=["--overwrite"])

In [ ]:
# Load pytorch file from a previous fit.
instances = [FitSnap(settings, arglist=["--overwrite"]) for _ in range(4)]
for i, inst in enumerate(instances):
    print(list_of_weights[i])
    t0 = time.time()
    inst.config.sections['PYTORCH'].save_state_input = list_of_weights[i]
    inst.solver.configs = configs
    (energies_model, forces_model) = inst.solver.evaluate_configs(config_idx=None, standardize_bool=True)
    t1 = time.time()
    total = t1-t0
    print('Total time (s) to obtain forces and energies from config file',total)

    with open("energies_model_"+str(i+1).zfill(2), "wb") as fp:   #Pickling
        pickle.dump(energies_model, fp)
        
    with open("forces_model_"+str(i+1).zfill(2), "wb") as fp:   #Pickling
        pickle.dump(forces_model, fp)
        
    del inst

In [ ]:
# import pickle

# with open("energies_model_"+str(i+1).zfill(2), "wb") as fp:   #Pickling
#     pickle.dump(energies_model, fp)
    
# with open("forces_model_"+str(i+1).zfill(2), "wb") as fp:   #Pickling
#     pickle.dump(forces_model, fp)


# # with open("energies_model_"+str(1).zfill(2), "rb") as fp:   # Unpickling
    
# #     b = pickle.load(fp)